# Import Libs

In [ ]:
%load_ext tensorboard

In [ ]:
import numpy as np
import pandas as pd
import datetime

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras


# Dataset Management functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/Tesi/ECG/dataset.zip -d dataset/

Archive:  drive/MyDrive/Tesi/ECG/dataset.zip
  inflating: dataset/X_test.npy      
  inflating: dataset/X_train.npy     
  inflating: dataset/y_test.csv      
  inflating: dataset/y_train.csv     


In [ ]:
data_path = "dataset/"

In [ ]:
def load_data(path):

  X_train = np.load(path + "X_train.npy")
  Y_train = pd.read_csv(path + "y_train.csv")

  X_test = np.load(path + "X_test.npy")
  Y_test = pd.read_csv(path + "y_test.csv")

  calsses = list(Y_train.columns)

  return X_train, Y_train.to_numpy(), X_test, Y_test.to_numpy(), calsses

# ResNet Model


In [ ]:
def get_resnet_multioutput(input_shape, classes):
    
    n_feature_maps = 32

    input_layer = keras.layers.Input(input_shape)

    # Block 1
    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # Block 2
    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # Block 3
    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # Final Block
    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
    
    outputs = []
    for c in classes:
      outputs.append(keras.layers.Dense(1, activation='sigmoid', name=c + "_out")(gap_layer))

    model = keras.models.Model(inputs=input_layer, outputs=outputs)
    
    return model

In [ ]:
def get_ECGNet(input_shape, classes):

    input_layer = keras.layers.Input(input_shape)

    outputs = []
    for c in classes:
      if c == "HYP":
        outputs.append(get_backbone(input_layer, c + "_out", 64))
      else:
        outputs.append(get_backbone(input_layer, c + "_out", 32))

    model = keras.models.Model(inputs=input_layer, outputs=outputs)
    
    return model

In [ ]:
def get_backbone(input, out_name, filters):

    n_feature_maps = filters

    # Block 1
    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input)
    conv_x = keras.ayers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # Block 2
    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # Block 3
    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # Final Block
    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

    output = keras.layers.Dense(1, activation='sigmoid', name=out_name)(gap_layer)
    
    return output

# Train and Test

In [ ]:
def train(model, train_X, train_Y, val_X, val_Y, epochs = 30, batch_size = 64, log_dir = "logs/train/"):

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                        histogram_freq=1)
  
  early_stop  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 patience=20)
  
  check_point = tf.keras.callbacks.ModelCheckpoint('best_model', 
                                                   monitor='val_loss', 
                                                   save_weights_only=True, 
                                                   save_best_only=True)
  
  history = model.fit(train_X, 
                      train_Y, 
                      epochs=epochs, 
                      batch_size = batch_size, 
                      validation_data = (val_X, val_Y),
                      callbacks=[tensorboard_callback, check_point],
                      verbose = 1)
  
  return history.history

In [ ]:
def test(model, test_X, test_y, batch_size = 64, plot_features = False):

  predictions = model.predict(test_X, batch_size=batch_size)
	
  return predictions

# Run Experiment

In [ ]:
%rm -rf ./logs/

In [ ]:
n_test = 2

X_train, Y_train, X_test, Y_test, classes = load_data(data_path)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

# Save the best model
best_score = 0
best_model = None
scores = []

# get the model and compile
if n_test == 1:
  model = get_resnet_multioutput((n_timesteps,n_features), classes)
else:
  model = get_ECGNet((n_timesteps,n_features), classes)
  
losses = {c + "_out": keras.losses.BinaryCrossentropy() for c in classes}
metrics = {c + "_out": [tf.keras.metrics.AUC(curve="PR", name=c + "_prauc"), 
                        tf.keras.metrics.AUC(name=c + "_rocauc"),
                        "accuracy"] for c in classes}

model.compile(loss=losses, optimizer='adam', metrics=metrics)

Y_train = [Y_train[:,i] for i in range(Y_train.shape[1])]
Y_val = [Y_val[:,i] for i in range(Y_val.shape[1])]
# train and test
history = train(model, X_train, Y_train, X_val, Y_val, epochs=20, log_dir = "logs/")
  
model.load_weights('best_model')

preds = model.predict(X_test, batch_size=64)


In [ ]:
np.save("drive/MyDrive/Tesi/ECG/preds{}".format(n_test), np.concatenate(preds, axis=1))
np.save("drive/MyDrive/Tesi/ECG/y{}".format(n_test), Y_test)

import pickle

with open('drive/MyDrive/Tesi/ECG/history{}.pkl'.format(n_test), "wb") as f:
  pickle.dump(history,f)

# Plot train

In [ ]:
import pickle
import matplotlib.pyplot as plt

with open('drive/MyDrive/Tesi/ECG/history{}.pkl'.format(n_test), 'rb') as f:
  history = pickle.load(f)

best_epoch = np.argmin(history["val_loss"])
val_pr_auc = 0
val_roc_auc = 0
val_acc = 0

for c in classes:
  val_pr_auc += history["val_{}_out_{}_prauc".format(c,c)][best_epoch]
  val_roc_auc += history["val_{}_out_{}_rocauc".format(c,c)][best_epoch]
  val_acc += history["val_{}_out_accuracy".format(c,c)][best_epoch]

val_pr_auc = val_pr_auc / len(classes)
val_roc_auc = val_roc_auc / len(classes)
val_acc = val_acc / len(classes)

print("ROC-AUC:",val_roc_auc)
print("PR-AUC:",val_pr_auc)
print("Accuracy:",val_acc)

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)         
plt.rc('axes', titlesize=BIGGER_SIZE)   
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE)

fig1, (ax1, ax2) = plt.subplots(1,2)
fig1.set_size_inches(18, 7)

x = np.arange(20) + 1

for c in classes:
  loss = history["val_{}_out_loss".format(c)]
  auc = history["val_{}_out_{}_prauc".format(c,c)]
  ax1.plot(x, loss, linewidth=2)
  ax2.plot(x, auc, linewidth=2)

ax1.set_title("Validation Loss")
ax2.set_title("Validation PR-AUC")

ax1.set_xlabel("epoch")
ax1.set_ylabel("loss")
ax2.set_xlabel("epoch")
ax2.set_ylabel("AUC (%)")
ax1.legend(['CD', 'HYP', 'MI', 'NORM', 'STTC'])
ax2.legend(['CD', 'HYP', 'MI', 'NORM', 'STTC'])
plt.show()

# Plot test

In [ ]:
n_test = 2

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

pred_test = np.load("drive/MyDrive/Tesi/ECG/preds{}.npy".format(n_test))
Y_test = np.load("drive/MyDrive/Tesi/ECG/y{}.npy".format(n_test))

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)         
plt.rc('axes', titlesize=BIGGER_SIZE)   
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE)

fig, ax = plt.subplots()
fig.set_size_inches(11, 10)

#pred_test = np.digitize(np.concatenate(preds, axis=1), [0.5])

classes = ['CD', 'HYP', 'MI', 'NORM', 'STTC']
colors = ['blue', 'orange', 'green', 'red', 'purple']

avg_prauc = 0
avg_rocauc = 0
avg_acc = 0
for i, (c, color) in enumerate(zip(classes, colors)):
    precision, recall, _ = precision_recall_curve(Y_test[:,i], pred_test[:,i])
    ax.plot(recall, precision, label=c, linewidth=3)
    avg_prauc += auc(recall, precision)
    avg_rocauc += roc_auc_score(Y_test[:,i], pred_test[:,i])
    avg_acc += accuracy_score(Y_test[:,i], (pred_test[:,i] > 0.5).astype(int))

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)

ax.set_title('PR Curve', fontsize=24)
ax.legend(classes)
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
plt.show()

print()
print("Avg. PR-AUC:", avg_prauc / len(classes))
print("Avg. ROC-AUC:", avg_rocauc / len(classes))
print("Avg. Accuracy:", avg_acc / len(classes))


In [ ]:
(pred_test > 0.5).astype(int)

# Tensor Board

In [ ]:
%tensorboard --logdir logs/